# Airfare Markets Under Pressure — EDA & Insights

**Objective:** Analyze when, where, and why airfares differ across U.S. domestic markets, and how city-level characteristics influence route-level prices. Deliver actionable insights for travelers, industry stakeholders, and policymakers.

**Data:** U.S. DOT Domestic Airline Consumer Airfare Report (2021–2025 Q2)

## 1. Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', 50)
plt.rcParams['figure.figsize'] = (10, 5)
sns.set_theme(style='whitegrid')

# Load Excel (requires: pip install pandas openpyxl)
DATA_PATH = 'airline_ticket_dataset.xlsx'
df = pd.read_excel(DATA_PATH, engine='openpyxl')
print('Shape:', df.shape)
print('\nColumns:', list(df.columns))
df.head(10)

In [ ]:
# Basic info and missing values
df.info()
print('\nMissing values:')
print(df.isnull().sum())

## 2. Data Cleaning & Column Mapping

Standardize column names for analysis (adjust if your dataset uses different names).

In [ ]:
# Inspect column names (run once, then map to standard names if needed)
for c in df.columns:
    print(repr(c), '->', df[c].dtype, '| sample:', df[c].dropna().iloc[0] if df[c].notna().any() else 'N/A')

In [ ]:
# Map common DOT report column names to a standard set (edit keys to match your actual columns)
col_map = {
    'Origin': 'origin', 'Origin City': 'origin_city', 'Origin Airport': 'origin',
    'Destination': 'dest', 'Destination City': 'dest_city', 'Destination Airport': 'dest',
    'Average Fare': 'avg_fare', 'Fare': 'avg_fare', 'Avg Fare': 'avg_fare', 'Average Fare ($)': 'avg_fare',
    'Passengers': 'passengers', 'Passenger Count': 'passengers', 'Total Passengers': 'passengers',
    'Quarter': 'quarter', 'Year': 'year', 'Year-Quarter': 'year_quarter',
    'Distance': 'distance', 'Miles': 'distance', 'Market Distance': 'distance',
    'Dominant Carrier': 'dominant_carrier', 'Dominant Carrier Share': 'dom_share', 'Market Share': 'dom_share',
    'Low Cost Carrier': 'lcc', 'LCC': 'lcc', 'LCC Penetration': 'lcc_penetration',
    'Lowest Fare Carrier': 'lowest_fare_carrier', 'Carriers': 'carriers', 'Number of Carriers': 'n_carriers',
}
# Only rename columns that exist
rename = {k: v for k, v in col_map.items() if k in df.columns}
if rename:
    df = df.rename(columns=rename)
print('Columns after mapping:', list(df.columns))
df.head(5)

## 3. When Do Fares Differ? — Temporal Patterns

In [ ]:
# Identify fare and time columns (use actual names from your data)
fare_col = next((c for c in df.columns if 'fare' in c.lower() or 'fare' in c), df.columns[0])
time_cols = [c for c in df.columns if any(x in c.lower() for x in ['year','quarter','date','q'])]
print('Fare column:', fare_col, '| Time-related columns:', time_cols)

# Aggregate average fare by year/quarter if available
if time_cols:
    agg = df.groupby(time_cols, dropna=False)[fare_col].agg(['mean','median','count']).reset_index()
    print(agg.head(15))
else:
    print('No obvious time column found. Show numeric columns:', df.select_dtypes(include=[np.number]).columns.tolist())

In [ ]:
# Plot: Average fare over time (adjust x/y to your column names)
if time_cols and fare_col:
    temporal = df.groupby(time_cols[0])[fare_col].mean().reset_index()
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(temporal[time_cols[0]].astype(str), temporal[fare_col], marker='o')
    ax.set_title('Average Airfare Over Time')
    ax.set_xlabel(time_cols[0])
    ax.set_ylabel(fare_col)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('Skipping temporal plot: need fare and time columns.')

## 4. Where Do Fares Differ? — Routes & City-Level Effects

In [ ]:
# Origin/destination columns
origin_col = next((c for c in df.columns if 'origin' in c.lower()), None)
dest_col = next((c for c in df.columns if 'dest' in c.lower() or 'destination' in c.lower()), None)
if not origin_col:
    origin_col = df.columns[0]
if not dest_col:
    dest_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]
print('Origin column:', origin_col, '| Dest column:', dest_col)

# Route-level average fare
route_cols = [c for c in [origin_col, dest_col] if c]
if len(route_cols) >= 2:
    route_fare = df.groupby(route_cols)[fare_col].agg(['mean','count']).reset_index()
    route_fare = route_fare[route_fare['count'] >= 10].sort_values('mean', ascending=False)
    print('Top 10 most expensive routes (min 10 obs):')
    print(route_fare.head(10))
    print('\nTop 10 cheapest routes:')
    print(route_fare.tail(10))

In [ ]:
# City-level: average fare when city is origin or destination
if origin_col and dest_col and fare_col:
    orig_fare = df.groupby(origin_col)[fare_col].mean().reset_index().rename(columns={origin_col: 'city', fare_col: 'avg_fare_origin'})
    dest_fare = df.groupby(dest_col)[fare_col].mean().reset_index().rename(columns={dest_col: 'city', fare_col: 'avg_fare_dest'})
    city_fare = orig_fare.merge(dest_fare, on='city', how='outer')
    city_fare['avg_fare'] = city_fare[['avg_fare_origin','avg_fare_dest']].mean(axis=1)
    city_fare = city_fare.nlargest(15, 'avg_fare')
    fig, ax = plt.subplots(figsize=(10,5))
    ax.barh(city_fare['city'].astype(str), city_fare['avg_fare'])
    ax.set_title('Cities with Highest Average Route Fares (origin/destination)')
    ax.set_xlabel('Average Fare')
    plt.tight_layout()
    plt.show()

## 5. Why Do Fares Differ? — Competition, Hub Dominance, LCC Penetration

In [ ]:
# Competition / dominance: correlate fare with market share, LCC, number of carriers
numeric = df.select_dtypes(include=[np.number]).columns.tolist()
print('Numeric columns:', numeric)
if fare_col in df.columns and len(numeric) > 1:
    corr = df[numeric].corr()[fare_col].drop(fare_col, errors='ignore').sort_values(key=abs, ascending=False)
    print('\nCorrelation with', fare_col, ':')
    print(corr.head(15))

In [ ]:
# Scatter: fare vs one key driver (e.g. distance or dominant share)
x_col = next((c for c in df.columns if 'distance' in c.lower() or 'mile' in c.lower()), None)
if not x_col and numeric:
    x_col = [c for c in numeric if c != fare_col]
    x_col = x_col[0] if x_col else None
if x_col and fare_col:
    plt.figure(figsize=(8,5))
    plt.scatter(df[x_col], df[fare_col], alpha=0.3, s=10)
    plt.xlabel(x_col)
    plt.ylabel(fare_col)
    plt.title(f'Fare vs {x_col}')
    plt.tight_layout()
    plt.show()

## 6. Predictive Model — Explaining Fare Variation

In [ ]:
# Prepare model matrix: numeric + encoded categoricals
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

model_df = df.copy()
for col in model_df.select_dtypes(include=['object']).columns:
    if model_df[col].nunique() < 100 and model_df[col].notna().sum() > 100:
        model_df[col] = LabelEncoder().fit_transform(model_df[col].astype(str))
    else:
        model_df = model_df.drop(columns=[col], errors='ignore')
model_df = model_df.select_dtypes(include=[np.number]).dropna(how='all', axis=1)
if fare_col not in model_df.columns:
    fare_col = next((c for c in df.columns if 'fare' in c.lower()), model_df.columns[0])
    model_df[fare_col] = df[fare_col]
y = model_df[fare_col]
X = model_df.drop(columns=[fare_col], errors='ignore').dropna(axis=1, how='all')
X = X.loc[:, X.nunique() > 1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = LinearRegression().fit(X_train, y_train)
pred = reg.predict(X_test)
print('MAE:', round(mean_absolute_error(y_test, pred), 2))
print('R²:', round(r2_score(y_test, pred), 4))
coef = pd.Series(reg.coef_, index=X.columns).sort_values(key=abs, ascending=False)
print('\nTop feature importance (coefficient magnitude):')
print(coef.head(15))

## 7. Summary & Actionable Insights

In [ ]:
print('''
--- ACTIONABLE INSIGHTS ---

1. TRAVELERS: Use route and time patterns above to prefer cheaper city-pairs and booking windows.
2. INDUSTRY: Hub dominance and LCC penetration drive fares; position on high-fare vs competitive routes.
3. POLICY: Monitor high-fare, low-competition markets for consumer protection and competition.
4. Re-run this notebook after aligning column names (Section 2) to your actual DOT dataset.
''')